In [1]:
# # setup for running on Google Colab
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# import os
# os.listdir("./drive/MyDrive/711/HW2")
# os.chdir("./drive/MyDrive/711/HW2")

Mounted at /content/drive


## RAG

REMOVED_TOKEN

### SetUp

In [ ]:
!pip install --quiet --upgrade langchain langchain-community langchain-chroma

In [ ]:
!pip install -U langchain langchain-openai

In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
os.environ["LANGCHAIN_API_KEY"] = "REMOVED_TOKEN"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "rag-template"
os.environ["USER_AGENT"] = "LangChain/1.0 (+https://www.langchain.com)"


In [5]:
from huggingface_hub import login

# Use your token here
# token = getpass.getpass("Enter your Hugging Face token: ")
# token = "REMOVED_TOKEN"
token = "REMOVED_TOKEN" # for llama3.1-8b-Instruct
login(token=token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Load models for embeddings and QA generation

In [5]:
!pip install sentence-transformers


  Obtaining dependency information for sentence-transformers from https://files.pythonhosted.org/packages/e7/78/2835963e7c03865392e88008629d9b81db386fedfcfac006d5c6454b8ecf/sentence_transformers-3.2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 12.5 MB/s eta 0:00:00


In [2]:
import os
from transformers import pipeline
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import torch

In [22]:
# Step 1: Initialize the Hugging Face model as your LLM
from transformers import AutoModelForCausalLM, AutoTokenizer
# Here using glaiveai/Llama-3-8B-RAG-v1
# model_name = "glaiveai/Llama-3-8B-RAG-v1"
# model_name = "unsloth/Llama-3.2-1B-Instruct"

model_name = "meta-llama/Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

generation_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer, 
    torch_dtype=torch.float16
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
# Step 2: Load the Sentence Transformers model for embeddings
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", truncate_dim=384)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Load Documents

In [6]:
# Step 3: Load docs
import os

def load_text_files(path):
    docs = []

    # Check if the path is a directory
    if os.path.isdir(path):
        # Iterate over files in the directory
        for file_name in os.listdir(path):
            if file_name.endswith(".txt"):
                file_path = os.path.join(path, file_name)
                with open(file_path, 'r', encoding='utf-8') as file:
                    docs.append(file.read())
    elif os.path.isfile(path) and path.endswith(".txt"):
        # If the path is a file, directly read it
        with open(path, 'r', encoding='utf-8') as file:
            docs.append(file.read())

    return docs

In [7]:
# docs = load_text_files("./data")
docs = load_text_files("../../data/crawled/crawled_text_data")

In [11]:
print(len(docs))

151


In [8]:
from langchain.docstore.document import Document
# Step 4: Split the documents into smaller chunks
# Wrap text strings in Document objects
documents = [Document(page_content=text) for text in docs]

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Split the documents into chunks
splits = text_splitter.split_documents(documents)

### Create vectorstore of embeddings

In [9]:
# Step 5: Create Chroma vectorstore with embeddings from Sentence Transformers
embeddings = [embedding_model.encode(doc.page_content) for doc in splits]

In [12]:
print("Dimension of embeddings:", len(embeddings[0]))
print("Number of embeddings:", len(embeddings))

Dimension of embeddings: 384
Number of embeddings: 2429


In [13]:
# Define a wrapper that provides the same interface as OpenAIEmbeddings
class SentenceTransformerEmbeddings:
    def __init__(self, model):
        self.model = model

    # Method to generate embeddings for a list of texts (documents)
    def embed_documents(self, texts):
        return [self.model.encode(text) for text in texts]

    # Method to generate embeddings for a single query (optional but useful)
    def embed_query(self, text):
        return self.model.encode(text)

In [14]:
# Assuming this is a fresh start, recreate the vectorstore
embedding_wrapper = SentenceTransformerEmbeddings(embedding_model)

# Create the Chroma vectorstore with the new 384-dimension embeddings
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_wrapper, collection_name="collectionChroma")


In [15]:
num_documents = vectorstore._collection.count()
num_documents

2429

### Prompt for QA generation

In [16]:
# Step 6: Set up the retriever and RAG prompt
from langchain import hub
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})
prompt = hub.pull("rlm/rag-prompt")

In [67]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate

# New template string
new_template = """
You are an expert assistant answering factual questions about Pittsburgh or Carnegie Mellon University (CMU). 
Use the retrieved information to give a detailed and helpful answer. If the provided context does not contain the answer, leverage your pretraining knowledge to provide the correct answer. 
If you truly do not know, just say "I don't know."

Important Instructions:
- Answer concisely without repeating the question.
- Use the provided context if relevant; otherwise, rely on your pretraining knowledge.
- Do **not** use complete sentences. Provide only the word, name, date, or phrase that directly answers the question. For example, given the question "When was Carnegie Mellon University founded?", you should only answer "1900".

Examples:
Question: Who is Pittsburgh named after? 
Answer: William Pitt
Question: What famous machine learning venue had its first conference in Pittsburgh in 1980? 
Answer: ICML
Question: What musical artist is performing at PPG Arena on October 13? 
Answer: Billie Eilish

Context: \n\n {context} \n\n
Question: {question} \n\n
Answer:
"""

# Create a new PromptTemplate
modified_prompt_template = PromptTemplate(
    input_variables=['context', 'question'],
    template=new_template
)

# Update the HumanMessagePromptTemplate with the new PromptTemplate
modified_human_message_template = HumanMessagePromptTemplate(prompt=modified_prompt_template)

# Update the ChatPromptTemplate with the modified message
modified_chat_prompt_template = ChatPromptTemplate(
    input_variables=['context', 'question'],
    messages=[modified_human_message_template]
)

# This is now your updated `ChatPromptTemplate` with the modified prompt
print(modified_chat_prompt_template)

prompt = modified_chat_prompt_template

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nYou are an expert assistant answering factual questions about Pittsburgh or Carnegie Mellon University (CMU). \nUse the retrieved information to give a detailed and helpful answer. If the provided context does not contain the answer, leverage your pretraining knowledge to provide the correct answer. \nIf you truly do not know, just say "I don\'t know."\n\nImportant Instructions:\n- Answer concisely without repeating the question.\n- Use the provided context if relevant; otherwise, rely on your pretraining knowledge.\n- Do **not** use complete sentences. Provide only the word, name, date, or phrase that directly answers the question. For example, given the question "When was Carnegie Mellon University founded?", you should only answer "1900".\n\nExamples:\nQuestion

In [69]:
# Retrieve context using the retriever
def format_retreived_docs(docs):
    """
    Format the retrieved documents as the following:
    Context 1: <content of document n>
    Context 2: <content of document n-1> # in reverse order
    ...
    """
    # reverse the order of the documents
    docs = reversed(docs)
    return "\n\n".join(f"Context {i + 1}: {doc.page_content}" for i, doc in enumerate(docs))


In [73]:
# read in the qa annotation csv
import pandas as pd
qa_df = pd.read_csv("../../data/annotated/generated_qa_pairs_3000_test20.csv")

questions = qa_df["Question"].tolist()
answers = qa_df["Answer"].tolist()

# random sample 10 qa pairs
import random
sample_size = 10
random.seed(221)
sample_indices = random.sample(range(len(questions)), sample_size)
sample_questions = [questions[i] for i in sample_indices]
sample_answers = [answers[i] for i in sample_indices]

In [74]:
generations = []

for question in sample_questions:
    # Retrieve documents based on the question
    retrieved_docs = retriever.invoke(question)

    # Format the documents
    context = format_retreived_docs(retrieved_docs)

    # Create the full prompt using the prompt template
    prompt_messages = prompt.format_messages(context=context, question=question)
    full_prompt = "\n".join(message.content for message in prompt_messages)
    
    print(full_prompt)
    
    messages = [
    {"role": "user", "content": full_prompt},
    ]
    with torch.no_grad():
        llm_output = generation_pipe(messages, max_new_tokens=50)
    
    generations.append(llm_output[0]["generated_text"][1]['content'])


You are an expert assistant answering factual questions about Pittsburgh or Carnegie Mellon University (CMU). 
Use the retrieved information to give a detailed and helpful answer. If the provided context does not contain the answer, leverage your pretraining knowledge to provide the correct answer. 
If you truly do not know, just say "I don't know."

Important Instructions:
- Answer concisely without repeating the question.
- Use the provided context if relevant; otherwise, rely on your pretraining knowledge.
- Do **not** use complete sentences. Provide only the word, name, date, or phrase that directly answers the question. For example, given the question "When was Carnegie Mellon University founded?", you should only answer "1900".

Examples:
Question: Who is Pittsburgh named after? 
Answer: William Pitt
Question: What famous machine learning venue had its first conference in Pittsburgh in 1980? 
Answer: ICML
Question: What musical artist is performing at PPG Arena on October 13? 
A

In [75]:
print(generations)
print(sample_answers)

['Pietrandrea', 'Preserving Underground Pittsburgh', 'La Traviata matinee', '2024', 'Robin Williams', 'St. Louis Blues', 'David L. Lawrence Convention Center', '4.5 seconds', 'Pennsylvania', '2,041,669']
['Monteverdi Society', 'Pittsburgh Jazz, Blues, and Bluegrass', 'La Traviata', '2024', 'Adrian Cronauer', 'New Jersey Devils', "The city's convention center", '4.5 seconds', 'Pennsylvania', '1,846,000']
